In [1]:
 pip install pennylane-cirq

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 1.8/1.8 MB 3.9 MB/s eta 0:00:00
     ---------------------------------------- 78.3/78.3 kB 4.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

In [3]:
import numpy as np
import pennylane as qml
import tensorflow as tf

In [4]:
dev = qml.device('cirq.simulator', wires=3)

In [5]:
def real(angles, **kwargs):
    qml.Hadamard(wires=0)
    qml.Rot(*angles, wires=0)

In [6]:
def generator(w, **kwargs):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=1)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RX(w[6], wires=0)
    qml.RY(w[7], wires=0)
    qml.RZ(w[8], wires=0)


def discriminator(w):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=2)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=2)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=2)
    qml.CNOT(wires=[0, 2])
    qml.RX(w[6], wires=2)
    qml.RY(w[7], wires=2)
    qml.RZ(w[8], wires=2)

In [7]:
@qml.qnode(dev)
def real_disc_circuit(phi, theta, omega, disc_weights):
    real([phi, theta, omega])
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))


@qml.qnode(dev)
def gen_disc_circuit(gen_weights, disc_weights):
    generator(gen_weights)
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))

In [9]:
def prob_real_true(disc_weights):
    true_disc_output = real_disc_circuit(phi, theta, omega, disc_weights)
    # convert to probability
    prob_real_true = (true_disc_output + 1) / 2
    return prob_real_true


def prob_fake_true(gen_weights, disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
    # convert to probability
    prob_fake_true = (fake_disc_output + 1) / 2
    return prob_fake_true


def disc_cost(disc_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights)
    return cost


def gen_cost(gen_weights):
    return -prob_fake_true(gen_weights, disc_weights)

In [10]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))
init_disc_weights = np.random.normal(size=(9,))

gen_weights = tf.Variable(init_gen_weights)
disc_weights = tf.Variable(init_disc_weights)

In [11]:
opt = tf.keras.optimizers.SGD(0.4)
opt.build([disc_weights, gen_weights])

In [16]:
!pip list

Package                      Version


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



---------------------------- --------------
absl-py                      2.1.0
annotated-types              0.6.0
anyio                        4.3.0
appdirs                      1.4.4
argon2-cffi                  23.1.0
argon2-cffi-bindings         21.2.0
arrow                        1.3.0
asttokens                    2.4.1
astunparse                   1.6.3
async-lru                    2.0.4
attrs                        23.2.0
autograd                     1.6.2
autoray                      0.6.9
Babel                        2.14.0
beautifulsoup4               4.12.3
bleach                       6.1.0
cachetools                   5.3.3
certifi                      2024.2.2
cffi                         1.16.0
charset-normalizer           3.3.2
cirq-core                    1.3.0
cirq-pasqal                  1.3.0
colorama                     0.4.6
comm                         0.2.2
contourpy                    1.2.1
cryptography                 42.0.5
cycler                       0.12.1

In [31]:
cost = lambda: disc_cost(disc_weights)

for step in range(50):
    opt.minimize(cost, [disc_weights])
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

AttributeError: 'SGD' object has no attribute 'minimize'

In [29]:
pip install pytorch


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pytorch
Failed to build pytorch
  Running setup.py install for pytorch: started
  Running setup.py install for pytorch: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\STUDENT.DESKTOP-AG6COQS\AppData\Local\Temp\pip-install-yfou4kav\pytorch_a61267720d1b4f27aad8db8e1f352798\setup.py", line 15, in <module>
          raise Exception(message)
      Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  error: subprocess-exited-with-error
  
  × Running setup.py install for pytorch did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-call

In [30]:


import torch.optim as optim

# Assuming you've defined your optimizer object 'opt' somewhere

cost = lambda: disc_cost(disc_weights)

for step in range(50):
    opt.zero_grad()
    loss = cost()
    loss.backward()
    opt.step()
    if step % 5 == 0:
        cost_val = cost().detach().numpy()
        print("Step {}: cost = {}".format(step, cost_val))


ModuleNotFoundError: No module named 'torch'

In [19]:


import tensorflow as tf

print(tf.__version__)

# Create an optimizer with the desired parameters.
opt = tf.keras.optimizers.SGD(learning_rate=0.4)
var1, var2 = tf.Variable(1.0), tf.Variable(2.0)
# `loss` is a callable that takes no argument and returns the value
# to minimize.
loss = lambda: 3 * var1 * var1 + 2 * var2 * var2
# Call minimize to update the list of variables.
opt.minimize(loss, var_list=[var1, var2])

2.16.1


AttributeError: 'SGD' object has no attribute 'minimize'

In [22]:
print("Prob(real classified as real): ", prob_real_true(disc_weights).numpy())

Prob(real classified as real):  0.18482935428619385


In [23]:
print("Prob(fake classified as real): ", prob_fake_true(gen_weights, disc_weights).numpy())

Prob(fake classified as real):  0.1623162031173706


In [24]:
cost = lambda: gen_cost(gen_weights)

for step in range(50):
    opt.minimize(cost, [gen_weights])
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

AttributeError: 'SGD' object has no attribute 'minimize'

In [25]:
print("Prob(fake classified as real): ", prob_fake_true(gen_weights, disc_weights).numpy())

Prob(fake classified as real):  0.1623162031173706


In [26]:
print("Discriminator cost: ", disc_cost(disc_weights).numpy())

Discriminator cost:  -0.022513151168823242


In [27]:
obs = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]

@qml.qnode(dev)
def bloch_vector_real(angles):
    real(angles)
    return [qml.expval(o) for o in obs]

@qml.qnode(dev)
def bloch_vector_generator(angles):
    generator(angles)
    return [qml.expval(o) for o in obs]

print(f"Real Bloch vector: {bloch_vector_real([phi, theta, omega])}")
print(f"Generator Bloch vector: {bloch_vector_generator(gen_weights)}")

Real Bloch vector: [array(-0.21694186), array(0.45048445), array(-0.86602533)]
Generator Bloch vector: [<tf.Tensor: shape=(), dtype=float64, numpy=0.022377027198672295>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0004875379672739655>, <tf.Tensor: shape=(), dtype=float64, numpy=-0.00974904652684927>]
